In [ ]:
import torch
import numpy as np
import matplotlib.pyplot as plt
import torchvision.transforms as T 

from torch import nn
from torch.utils.data import DataLoader 
from torchvision import datasets 
from sklearn.metrics import accuracy_score

In [2]:
my_transform = T.Compose([
    T.ToTensor(),
    T.Normalize((0.5,),(0.5,))
])

In [3]:
# Loading Dataset
train_dataset = datasets.MNIST(root = './data', train = True, download = False, transform = my_transform )
test_dataset = datasets.MNIST(root = './data', train = False, download = False, transform = my_transform )

In [4]:
# DataLoader
train_loader = DataLoader(dataset = train_dataset, batch_size = 64, shuffle = True)
test_loader = DataLoader(dataset = test_dataset, batch_size = 1000, shuffle = False)

In [5]:
# Flatten the image to [784]
train_dataset[0][0].view(1,-1).shape


torch.Size([1, 784])

In [6]:
class ANNModel(nn.Module):
    def __init__(self):
        super(ANNModel, self).__init__()
        self.fc1 = nn.Linear(28*28,512)
        self.fc2 = nn.Linear(512,128)
        self.fc3 = nn.Linear(128,10)
    def forward(self,x):
        x = x.view(x.size(0), -1)
        x = torch.relu(self.fc1(x))
        x = torch.relu(self.fc2(x))
        x = self.fc3(x)
        return x

In [7]:
model = ANNModel().to("cuda")

In [8]:
criterian = nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(model.parameters(), lr = 0.001)
epochs = 10
device = "cuda"
scheduler = torch.optim.lr_scheduler.StepLR(
    optimizer, 
    step_size=5,  # Reduce LR every 5 epochs
    gamma=0.9     # Multiply LR by 0.5 each time
)


In [9]:
def evaluate(test_loader):
    with torch.no_grad():
        test_loss_overall = 0.0
        true_label = []
        pred_label = []
        for data, label in test_loader:
            data, label = data.to(device), label.to(device)
            test_pred = model(data)
            test_loss = criterian(test_pred, label)
            test_loss_overall += test_loss.item()
            _ , test_prediction = torch.max(test_pred, 1)
            true_label.append(label.cpu().numpy())
            pred_label.append(test_prediction.cpu().numpy())
        average_test_loss = test_loss_overall/len(test_loader)
        ty_true = np.concatenate(true_label)
        ty_pred = np.concatenate(pred_label)

        test_accuracy = accuracy_score(ty_true, ty_pred)
        print(f'')
        return average_test_loss, test_accuracy


In [10]:
def train(epochs: int, train_data_loader: DataLoader, path: str):
    prev_test_accuracy = 0
    for epoch in range(epochs):
        train_loss = 0.0
        pred_label = []
        actual_labels = []
        model.train()

        for batch, (data, label) in enumerate(train_data_loader):
            data, label = data.to(device), label.to(device)
            model.zero_grad()
            pred = model(data)
            loss = criterian(pred, label)
            loss.backward()
            optimizer.step()
            train_loss += loss.item()
            _, prediction = torch.max(pred, 1)
            actual_labels.append(label.cpu().numpy())
            pred_label.append(prediction.cpu().detach().numpy())
        scheduler.step()
        average_loss = train_loss/len(train_loader)
        y_pred = np.concatenate(pred_label)
        y_actual = np.concatenate(actual_labels)
        accuracy = accuracy_score(y_actual, y_pred)
        test_average_loss, test_accuracy = evaluate(test_loader)
        print(f'Epoch {epoch+1} : Train Accuracy: {accuracy}, train loss: {loss}, Testing result: Test Accuracy: {test_accuracy}, Test Loss: {test_average_loss} ')
        if prev_test_accuracy < test_accuracy:
            prev_test_accuracy = test_accuracy
            torch.save(model.state_dict(),path)
            print(f"Model is saved at Test Accuracy of: {test_accuracy}")
        


In [11]:
path = r'D:\BOOK\Deep_learning_book\ANN\Models\MNIST.pt'

In [12]:
train(epochs = epochs, train_data_loader = train_loader, path = path)


Epoch 1 : Train Accuracy: 0.9026833333333333, train loss: 0.2016885131597519, Testing result: Test Accuracy: 0.9475, Test Loss: 0.16532125435769557 
Model is saved at Test Accuracy of: 0.9475

Epoch 2 : Train Accuracy: 0.9572333333333334, train loss: 0.05547843128442764, Testing result: Test Accuracy: 0.9445, Test Loss: 0.1781404621899128 

Epoch 3 : Train Accuracy: 0.9673166666666667, train loss: 0.07282276451587677, Testing result: Test Accuracy: 0.9675, Test Loss: 0.09913906045258045 
Model is saved at Test Accuracy of: 0.9675

Epoch 4 : Train Accuracy: 0.9727, train loss: 0.02198299951851368, Testing result: Test Accuracy: 0.9683, Test Loss: 0.09943986497819424 
Model is saved at Test Accuracy of: 0.9683

Epoch 5 : Train Accuracy: 0.97715, train loss: 0.28867852687835693, Testing result: Test Accuracy: 0.9726, Test Loss: 0.09162338636815548 
Model is saved at Test Accuracy of: 0.9726

Epoch 6 : Train Accuracy: 0.9811666666666666, train loss: 0.04065467789769173, Testing result: Te